In [1]:
batch_size = int(2e5)
# batch_size_score = int(1e5) #int(2e5)
executor_instances = 10
max_batch_size = int(2e7)
max_score_batch_size = int(5e5)
baucket_size = int(11e6) #8.5\
executor_cores = 2
executor_memory = 8

import os
import sys

if 'sc' in globals(): sc.stop()

sys.path.insert(0, os.path.abspath('../../../utils'))
spark_home = '/opt/cloudera/parcels/SPARK2/lib/spark2'
os.environ['SPARK_HOME'] = spark_home
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/venvs/anaconda/bin/python'
os.environ['PYSPARK_PYTHON'] = '/opt/venvs/anaconda/bin/python'
os.environ['LD_LIBRARY_PATH'] = '/opt/python/virtualenv/jupyter/lib'
sys.path.insert(0, os.path.join (spark_home,'python'))
sys.path.insert(0, os.path.join (spark_home,'python/lib/py4j-0.10.7-src.zip'))
from pyspark import SparkContext, SparkConf

In [2]:

conf = SparkConf().setAppName('data_prep').\
    setMaster("yarn-client").\
    set('spark.yarn.appMasterEnv.ARROW_PRE_0_15_IPC_FORMAT', 1).\
    set('spark.executorEnv.ARROW_PRE_0_15_IPC_FORMAT',1).\
    set('spark.local.dir', 'sparktmp').\
    set('spark.executor.memory','{}g'.format(executor_memory)).\
    set('spark.executor.instances', '{}'.format(executor_instances)).\
    set('spark.port.maxRetries', '500').\
    set('spark.executor.cores', '{}'.format(executor_cores)).\
    set('spark.dynamicAllocation.enabled', 'false').\
    set('spark.network.timeout', '700').\
    set("spark.sql.execution.arrow.enabled","true").\
    set("spark.kryoserializer.buffer.max", '2047mb').\
    set("spark.driver.memory",'13g').\
    set("spark.driver.cores", '9').\
    set("spark.dynamicAllocation.enabled", "false").\
    set("hive.exec.dynamic.partition", "true").\
    set("hive.exec.dynamic.partition.mode", "nonstrict").\
    set("spark.sql.execution.arrow.maxRecordsPerBatch", '{}'.format(batch_size)).\
    set("spark.sql.sources.bucketing.maxBuckets", '{}'.format(baucket_size)).\
    set("have.merge.sparkfiles", "true").\
    set("spark.files.overwrite", "true")
    
#set('spark.sql.execution.arrow.fallback.enabled', 'false')
#set('spark.worker.cleanup.enabled', 'true') #тогда не будут сохраняться файлы
#set('spark.driver.maxResultSize','10g').\
# set("hive.exec.dynamic.partition", "true").\
# set("hive.exec.dynamic.partition.mode", "nonstrict")
# 'spark.sql.execution.arrow.fallback.enabled' is set to true.

sc = SparkContext.getOrCreate(conf=conf).addFile(f'{os.getcwd()}/dspl.zip')
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.0.cloudera2
      /_/

Using Python version 3.7.2 (default, Dec 29 2018 06:19:36)
SparkSession available as 'spark'.


In [3]:
import datetime as dt, gc, pandas as pd, numpy as np, pickle, joblib
from tqdm import tqdm
from dspl.feature_extraction import transformers
exec(open('../cltv_spark_and_hive_funcs/cltv_spark_and_hive_funcs.py').read())
r = read_hive_table

In [7]:
config = {}

In [8]:
config["categorical_features"] = ['seg_age_segment', 'Product', 'sd_gender_cd', 'sd_mdm_status_cd', 'sd_name_age_segment_cd', 'sd_stlmnt_type_cd', 'sd_stlmnt_type2_cd', 'seg_crm_segment_cd', 'seg_client_ret_segment_cd', 'seg_calc_segment_cd', 'seg_client_segment_cd', 'seg_client_segment_prev_cd', 'prd_lst_prod_tb_cd', 'prd_lst_prod_osb_cd', 'prd_lst_prod_vsp_cd', 'prd_lst_prod_cd', 'prd_1st_prod_tb_cd', 'prd_1st_prod_osb_cd', 'prd_1st_prod_vsp_cd', 'prd_1st_prod_cd', 'prd_client_type_final_cd', 'prd_client_type_final_prev_cd', 'prd_activity_status_cd', 'prd_activity_status_prev_cd', 'lbt_acct_max_bal_tb_cd', 'lbt_acct_max_bal_osb_cd', 'lbt_acct_max_bal_vsp_cd', 'crd_lst_agrmnt_tb_cd', 'crd_lst_agrmnt_osb_cd', 'crd_lst_agrmnt_vsp_cd', 'cred_cred_hist_cd', 'tp_active_kind_cd', 'tp_best_kind_ever_cd', 'tp_lst_tb_cd', 'tp_lst_kind_cd', 'prd_activity_status_1m_cd', 'prd_activity_status_prev_1m_cd', 'lbt_payroll_tax_payer_cd', 'seg_service_channel_cd', 'sd_name_age_segment2_cd', 'cmpn_dm_addr_src_cd', 'cmpn_email_src_cd', 'cmpn_seg_client_ret_segment_cd', 'cmpn_sms_operator_net_cd', 'crd_cc_lst_ips_cd', 'crd_dc_lst_ips_cd', 'seg_crd_segm_special_cd', 'seg_crd_segm_special_prev_cd', 'seg_premier_cd', 'sd_client_valid_nflag', 'sd_doc_valid_nflag', 'sd_russian_citizen_nflag', 'sd_resident_nflag', 'sd_sbrf_employee_nflag', 'seg_private_banking_nflag', 'prd_dc_active_nflag', 'prd_dc_active_prev_nflag', 'prd_cc_active_nflag', 'prd_cc_active_prev_nflag', 'prd_ma_active_nflag', 'prd_ma_active_prev_nflag', 'prd_da_active_nflag', 'prd_da_active_prev_nflag', 'prd_sc_active_nflag', 'prd_sc_active_prev_nflag', 'prd_pl_active_nflag', 'prd_pl_active_prev_nflag', 'prd_mg_active_nflag', 'prd_mg_active_prev_nflag', 'prd_ib_active_nflag', 'prd_ib_active_prev_nflag', 'prd_mb_active_nflag', 'prd_mb_active_prev_nflag', 'prd_ap_cell_active_nflag', 'prd_ap_cell_active_prev_nflag', 'prd_ap_utl_active_nflag', 'prd_ap_utl_active_prev_nflag', 'prd_ap_tlc_active_nflag', 'prd_ap_tlc_active_prev_nflag', 'prd_ap_loan_active_nflag', 'prd_ap_loan_active_prev_nflag', 'prd_ap_budg_active_nflag', 'prd_ap_budg_active_prev_nflag', 'prd_ap_othr_active_nflag', 'prd_ap_othr_active_prev_nflag', 'prd_ap_cnbx_active_nflag', 'prd_ap_cnbx_active_prev_nflag', 'prd_ap_p2p_active_nflag', 'prd_ap_p2p_active_prev_nflag', 'prd_client_active_nflag', 'prd_client_active_prev_nflag', 'lbt_payroll_client_nflag', 'lbt_social_client_nflag', 'crd_dc_payroll_nflag', 'crd_dc_payroll_pmt_nflag', 'crd_dc_payroll_act_nflag', 'crd_dc_payroll_sbrf_act_nflag', 'crd_dc_social_nflag', 'crd_dc_social_pmt_nflag', 'crd_dc_social_act_nflag', 'crd_cc_overdue_nflag', 'crd_dc_ever_nflag', 'crd_cc_ever_nflag', 'cred_overdue_nflag', 'cred_intr_overdue_nflag', 'srv_thanks_nflag', 'srv_mb_nflag', 'srv_ap_cell_nflag', 'srv_ap_utl_nflag', 'srv_ap_tlc_nflag', 'srv_ap_loan_nflag', 'srv_ap_budg_nflag', 'srv_ap_othr_nflag', 'srv_ap_cnbx_nflag', 'srv_ap_p2p_nflag', 'srv_udbo_nflag', 'ext_stop_list_nflag', 'ext_dead_nflag', 'tp_active_nflag', 'tp_ever_nflag', 'prd_dc_active_1m_nflag', 'prd_cc_active_1m_nflag', 'prd_ib_active_1m_nflag', 'prd_mb_active_1m_nflag', 'prd_ap_cell_active_1m_nflag', 'prd_ap_utl_active_1m_nflag', 'prd_ap_tlc_active_1m_nflag', 'prd_ap_loan_active_1m_nflag', 'prd_ap_budg_active_1m_nflag', 'prd_ap_othr_active_1m_nflag', 'prd_ap_cnbx_active_1m_nflag', 'prd_ap_p2p_active_1m_nflag', 'prd_client_active_1m_nflag', 'prd_mf_active_nflag', 'prd_mf_active_prev_nflag', 'srv_sbol_1m_login_nflag', 'srv_sbol_3m_login_nflag', 'srv_sbol_web_1m_login_nflag', 'srv_sbol_web_3m_login_nflag', 'srv_sbol_mob_1m_login_nflag', 'srv_sbol_mob_3m_login_nflag', 'srv_sbol_atm_1m_login_nflag', 'srv_sbol_atm_3m_login_nflag', 'seg_mvs_nflag', 'crd_open_nflag', 'crd_ever_nflag', 'ppc_crd_dc_active_1m_nflag', 'ppc_crd_dc_active_nflag', 'ppc_crd_cc_active_1m_nflag', 'ppc_crd_cc_active_nflag', 'ppc_mb_active_1m_nflag', 'ppc_mb_active_nflag', 'ppc_p2p_active_1m_nflag', 'ppc_p2p_active_nflag', 'ppc_pay_active_1m_nflag', 'ppc_pay_active_nflag', 'lne_ovd_overdue_nflag', 'cmpn_cb_staff_nflag', 'cla_all_active_nflag', 'cred_overdue_nflag_3m', 'cred_intr_overdue_nflag_3m', 'cred_overdue_nflag_6m', 'cred_intr_overdue_nflag_6m', 'cred_overdue_nflag_12m', 'cred_intr_overdue_nflag_12m', 'cmpn_blogger_nflag', 'cmpn_cell_phone_mbk_nflag', 'cmpn_client_contact_nflag', 'cmpn_client_eligible_nflag', 'cmpn_comm_champion_nflag', 'cmpn_control_group_nflag', 'cmpn_cross_chnl_clnt_nflag', 'cmpn_dm_avail_nflag', 'cmpn_doc_fms_invalid_nflag', 'cmpn_doc_unique_nflag', 'cmpn_email_avail_nflag', 'cmpn_friends_nflag', 'cmpn_householder_VIP_nflag', 'cmpn_mdm_bad_fio_nflag', 'cmpn_mdm_bad_gender_nflag', 'cmpn_mdm_cell_phone_mbk_nflag', 'cmpn_mdm_sms_avail_nflag', 'cmpn_nflag_cbc_stop_list_nflag', 'cmpn_nflag_empl_16_plus', 'cmpn_nflag_empl_DZO_flag', 'cmpn_portf_control_group_nflag', 'cmpn_potential_vip_nflag', 'cmpn_public_person_nflag', 'cmpn_sms_avail_nflag', 'cmpn_stop_list_nflag', 'cmpn_tm_avail_nflag', 'cmpn_tpin_avail_nflag', 'cmpn_true_stop_list_nflag', 'crd_cc_open_mc_nflag', 'crd_cc_open_mir_nflag', 'crd_cc_open_visa_nflag', 'crd_dc_open_mc_nflag', 'crd_dc_open_mir_nflag', 'crd_dc_open_visa_nflag', 'crd_mc_open_nflag', 'crd_visa_open_nflag', 'cred_coborrower_nflag', 'cred_cred_hist_bad_nflag', 'cred_cred_hist_good_nflag', 'cred_cred_hist_med_nflag', 'cred_cred_hist_none_nflag', 'cred_litigation_nflag', 'cred_open_nflag', 'cred_restructured_nflag', 'dep_pop_act_nflag', 'dep_pop_exp_2m_nflag', 'dep_pop_exp_3m_nflag', 'dep_sohr_exp_2m_nflag', 'dep_sohr_exp_3m_nflag', 'dep_upr_exp_2m_nflag', 'dep_upr_exp_3m_nflag', 'seg_prd_ap_avail_nflag', 'seg_prd_mb_full_avail_nflag', 'seg_prd_mb_otd_avail_nflag', 'seg_prd_mb_own_avail_nflag', 'seg_prd_mb_p2p_avail_nflag', 'seg_prd_sbol_tip_avail_nflag', 'seg_prd_sbol_web_avail_nflag', 'seg_prd_smart_avail_nflag', 'seg_prd_uko_avail_nflag', 'srv_ap_cell_dc_nflag', 'srv_mb_dc_nflag', 'srv_mb_quick_service_nflag', 'prd_dc_activity_ind', 'prd_cc_activity_ind', 'prd_ma_activity_ind', 'prd_da_activity_ind', 'prd_sc_activity_ind', 'prd_pl_activity_ind', 'prd_mg_activity_ind', 'prd_ib_activity_ind', 'prd_mb_activity_ind', 'prd_ap_cell_activity_ind', 'prd_ap_utl_activity_ind', 'prd_ap_tlc_activity_ind', 'prd_ap_loan_activity_ind', 'prd_ap_budg_activity_ind', 'prd_ap_othr_activity_ind', 'prd_ap_cnbx_activity_ind', 'prd_ap_p2p_activity_ind', 'prd_client_activity_ind', 'prd_dc_activity_1m_ind', 'prd_cc_activity_1m_ind', 'prd_ib_activity_1m_ind', 'prd_mb_activity_1m_ind', 'prd_ap_cell_activity_1m_ind', 'prd_ap_utl_activity_1m_ind', 'prd_ap_tlc_activity_1m_ind', 'prd_ap_loan_activity_1m_ind', 'prd_ap_budg_activity_1m_ind', 'prd_ap_othr_activity_1m_ind', 'prd_ap_cnbx_activity_1m_ind', 'prd_ap_p2p_activity_1m_ind', 'prd_client_activity_1m_ind', 'prd_mf_activity_ind', 'cmpn_prev_seg_crd_trx_segm', 'cmpn_prev_seg_crd_trx_subsegm', 'cmpn_random_group1_ind', 'cmpn_seg_crd_pos_cat_group', 'cmpn_seg_crd_trx_segm', 'cmpn_seg_crd_trx_subsegm', 'cmpn_trx_segm_activity_ind', 'cmpn_trx_subsegm_activity_ind', 'cred_cred_hist_auto_max_ind', 'cred_cred_hist_auto_min_ind', 'cred_cred_hist_cc_max_ind', 'cred_cred_hist_cc_min_ind', 'cred_cred_hist_mg_max_ind', 'cred_cred_hist_mg_min_ind', 'cred_cred_hist_othr_max_ind', 'cred_cred_hist_othr_min_ind', 'cred_cred_hist_pl_max_ind', 'cred_cred_hist_pl_min_ind', 'cred_cred_hist_tot_max_ind', 'cred_cred_hist_tot_min_ind', 'ap', 'cc', 'ch', 'd33', 'dc', 'dep', 'dlcc', 'dnpr', 'h', 'hu', 'iis', 'mapp', 'mb', 'ph', 'pl', 'plg', 'pt', 'sbl', 'sc', 'sf', 'sh', 'sm', 'st', 't2f', 'th', 'seg_client_mp_segment_cd', 'prl_tax_payer_id', 'prd_crd_dc_active_nflag', 'prd_crd_cc_active_nflag', 'dep_payroll_client_nflag', 'dep_social_client_nflag', 'lne_loan_overdue_nflag', 'lne_loan_overdue_nflag', 'sd_dead_nflag', 'prd_crd_dc_active_1m_nflag', 'prd_crd_cc_active_1m_nflag', 'prd_crd_dc_active_1m_nflag', 'prd_crd_dc_active_nflag', 'prd_crd_cc_active_1m_nflag', 'prd_crd_cc_active_nflag', 'prd_p2p_active_1m_nflag', 'prd_p2p_active_nflag', 'prd_pay_active_1m_nflag', 'prd_pay_active_nflag', 'lne_overdraft_overdue_nflag', 'sd_cb_staff_nflag']

config["drop_features"] = ['client_dk', 'report_dt','sd_doc_type_id', 'seg_crm_client_mgr_login_name', 'seg_crd_txn_mcc_cluster_id', 'lbt_lst_payroll_agrmnt_id', 'cmpn_doc_type_id', 'cmpn_loyal_control_group_id', 'cmpn_sms_operator_id', 'crd_cc_lst_plastic_type_id', 'crd_dc_lst_plastic_type_id', 'crd_main_card_dwh_id', 'prd_lst_prod_division_dk', 'prd_1st_prod_division_dk', 'lbt_acct_max_bal_division_dk', 'lbt_acct_lst_division_dk', 'lbt_acct_1st_division_dk', 'ext_stop_list_reason_rk', 'tp_lst_division_dk', 'lbt_payroll_agrmnt_nbr', 'sd_doc_type_name', 'seg_crd_txn_mcc_cluster_name', 'prd_lst_prod_tb_name', 'prd_1st_prod_tb_name', 'ext_stop_list_reason_name', 'crd_cc_mnth_snc_issue_frac', 'lbt_social_disab_pension', 'prd_city_lst_prod', 'ext_employee_dzo', 'cmpn_ctzshp_cntry_name', 'cmpn_sms_city_name', 'cmpn_sms_dlvr_time_intr_name', 'cmpn_sms_tz_msk_shift', 'cmpn_sms_tz_utc_shift', 'crd_cc_lst_plastic_type_name', 'crd_dc_lst_plastic_type_name', 'dep_lbt_acct_lst_division_dk', 'lbt_payroll_tax_payer_cd', 'dep_acct_max_bal_division_dk', 'dep_lst_payroll_agrmnt_id', 'dep_acct_lst_division_dk', 'dep_acct_1st_division_dk', 'prl_tax_payer_id', 'prl_agrmnt_nbr', 'prl_social_disab_pension_nflag', 'prl_employee_dzo_nflag']

In [9]:
allowed_features = ['sd_gender_cd', 'sd_client_valid_nflag', 'sd_age_yrs_frac_nv', 'sd_age_yrs_comp_nv', 'sd_name_age_segment_cd', 'sd_stlmnt_type_cd', 'sd_doc_valid_nflag', 'sd_russian_citizen_nflag', 'sd_resident_nflag', 'sd_sbrf_employee_nflag', 'seg_crm_segment_cd', 'seg_private_banking_nflag', 'prd_lst_prod_tb_cd', 'prd_lst_prod_cd', 'prd_1st_prod_tb_cd', 'prd_1st_prod_cd', 'prd_ma_active_nflag', 'prd_ma_active_qty', 'prd_da_active_nflag', 'prd_da_active_qty', 'prd_sc_active_nflag', 'prd_pl_active_nflag', 'prd_pl_active_qty', 'prd_mg_active_nflag', 'prd_mg_active_qty', 'prd_ib_active_nflag', 'prd_ib_active_qty', 'prd_mb_active_nflag', 'prd_mb_active_qty', 'prd_ap_cell_active_nflag', 'prd_ap_cell_active_qty', 'prd_ap_utl_active_nflag', 'prd_ap_utl_active_qty', 'prd_ap_tlc_active_nflag', 'prd_ap_tlc_active_qty', 'prd_ap_loan_active_nflag', 'prd_ap_loan_active_qty', 'prd_ap_budg_active_nflag', 'prd_ap_budg_active_qty', 'prd_ap_othr_active_nflag', 'prd_ap_othr_active_qty', 'prd_ap_cnbx_active_nflag', 'prd_ap_cnbx_active_qty', 'prd_ap_p2p_active_nflag', 'prd_ap_p2p_active_qty', 'prd_client_active_nflag', 'prd_client_active_prod_qty', 'prd_client_type_final_cd', 'crd_dc_payroll_nflag', 'crd_dc_payroll_pmt_nflag', 'crd_dc_payroll_act_nflag', 'crd_dc_payroll_sbrf_act_nflag', 'crd_dc_social_nflag', 'crd_dc_social_pmt_nflag', 'crd_dc_social_act_nflag', 'crd_dc_open_qty', 'crd_dc_act_qty', 'crd_dc_act_spend_qty', 'crd_cc_open_qty', 'crd_cc_issued_qty', 'crd_cc_act_qty', 'crd_cc_act_spend_qty', 'crd_tot_act_qty', 'crd_tot_act_spend_qty', 'crd_cc_tot_lim_rub_amt', 'crd_cc_own_bal_rub_amt', 'crd_cc_tot_os_rub_amt', 'crd_cc_tot_credit_rub_amt', 'crd_cc_overdue_nflag', 'crd_cc_tot_otb_rub_amt', 'crd_cc_utilization_pct', 'crd_inf_total_rub_amt', 'crd_inf_dc_total_rub_amt', 'crd_inf_cc_total_rub_amt', 'crd_inf_total_qty', 'crd_inf_dc_total_qty', 'crd_inf_cc_total_qty', 'crd_inf_ext_transf_rub_amt', 'crd_inf_ext_transf_qty', 'crd_inf_cash_adv_rub_amt', 'crd_inf_cash_adv_qty', 'crd_inf_transfer_rub_amt', 'crd_inf_transfer_qty', 'crd_inf_intr_rub_amt', 'crd_inf_intr_qty', 'crd_inf_pos_ret_rub_amt', 'crd_inf_pos_ret_qty', 'crd_inf_othr_rub_amt', 'crd_inf_othr_qty', 'crd_otf_total_rub_amt', 'crd_otf_dc_total_rub_amt', 'crd_otf_cc_total_rub_amt', 'crd_otf_total_qty', 'crd_otf_dc_total_qty', 'crd_otf_cc_total_qty', 'crd_otf_pos_spend_rub_amt', 'crd_otf_dc_pos_spend_rub_amt', 'crd_otf_cc_pos_spend_rub_amt', 'crd_otf_pos_spend_qty', 'crd_otf_dc_pos_spend_qty', 'crd_otf_cc_pos_spend_qty', 'crd_otf_fee_rub_amt', 'crd_otf_fee_qty', 'crd_otf_cash_rub_amt', 'crd_otf_dc_cash_rub_amt', 'crd_otf_cc_cash_rub_amt', 'crd_otf_cash_qty', 'crd_otf_dc_cash_qty', 'crd_otf_cc_cash_qty', 'crd_otf_cash_sb_rub_amt', 'crd_otf_cash_sb_qty', 'crd_otf_cash_ext_rub_amt', 'crd_otf_cash_ext_qty', 'crd_otf_loan_rub_amt', 'crd_otf_loan_qty', 'crd_otf_trnf_org_rub_amt', 'crd_otf_trnf_org_qty', 'crd_otf_trnf_pers_rub_amt', 'crd_otf_trnf_pers_qty', 'crd_otf_trnf_pers_sbol_rub_amt', 'crd_otf_trnf_pers_sbol_qty', 'crd_otf_trnf_pers_mb_rub_amt', 'crd_otf_trnf_pers_mb_qty', 'crd_otf_trnf_pers_pos_rub_amt', 'crd_otf_trnf_pers_pos_qty', 'crd_otf_trnf_pers_atm_rub_amt', 'crd_otf_trnf_pers_atm_qty', 'crd_otf_othr_rub_amt', 'crd_otf_othr_qty', 'crd_pos_net_rub_amt', 'crd_dc_pos_auto_qty', 'crd_dc_pos_auto_rub_amt', 'crd_dc_pos_money_trf_qty', 'crd_dc_pos_money_trf_rub_amt', 'crd_dc_pos_eat_out_qty', 'crd_dc_pos_eat_out_rub_amt', 'crd_dc_pos_utilities_qty', 'crd_dc_pos_utilities_rub_amt', 'crd_dc_pos_pc_it_qty', 'crd_dc_pos_pc_it_rub_amt', 'crd_dc_pos_beuaty_qty', 'crd_dc_pos_beuaty_rub_amt', 'crd_dc_pos_healthcare_qty', 'crd_dc_pos_healthcare_rub_amt', 'crd_dc_pos_clothes_qty', 'crd_dc_pos_clothes_rub_amt', 'crd_dc_pos_leisure_qty', 'crd_dc_pos_leisure_rub_amt', 'crd_dc_pos_groceries_qty', 'crd_dc_pos_groceries_rub_amt', 'crd_dc_pos_special_qty', 'crd_dc_pos_special_rub_amt', 'crd_dc_pos_telecom_qty', 'crd_dc_pos_telecom_rub_amt', 'crd_dc_pos_home_repair_qty', 'crd_dc_pos_home_repair_rub_amt', 'crd_dc_pos_tourism_qty', 'crd_dc_pos_tourism_rub_amt', 'crd_dc_pos_dept_stores_qty', 'crd_dc_pos_dept_stores_rub_amt', 'crd_dc_pos_othr_qty', 'crd_dc_pos_othr_rub_amt', 'crd_cc_pos_auto_qty', 'crd_cc_pos_auto_rub_amt', 'crd_cc_pos_money_trf_qty', 'crd_cc_pos_money_trf_rub_amt', 'crd_cc_pos_eat_out_qty', 'crd_cc_pos_eat_out_rub_amt', 'crd_cc_pos_utilities_qty', 'crd_cc_pos_utilities_rub_amt', 'crd_cc_pos_pc_it_qty', 'crd_cc_pos_pc_it_rub_amt', 'crd_cc_pos_beuaty_qty', 'crd_cc_pos_beuaty_rub_amt', 'crd_cc_pos_healthcare_qty', 'crd_cc_pos_healthcare_rub_amt', 'crd_cc_pos_clothes_qty', 'crd_cc_pos_clothes_rub_amt', 'crd_cc_pos_leisure_qty', 'crd_cc_pos_leisure_rub_amt', 'crd_cc_pos_groceries_qty', 'crd_cc_pos_groceries_rub_amt', 'crd_cc_pos_special_qty', 'crd_cc_pos_special_rub_amt', 'crd_cc_pos_telecom_qty', 'crd_cc_pos_telecom_rub_amt', 'crd_cc_pos_home_repair_qty', 'crd_cc_pos_home_repair_rub_amt', 'crd_cc_pos_tourism_qty', 'crd_cc_pos_tourism_rub_amt', 'crd_cc_pos_dept_stores_qty', 'crd_cc_pos_dept_stores_rub_amt', 'crd_cc_pos_othr_qty', 'crd_cc_pos_othr_rub_amt', 'crd_lst_agrmnt_tb_cd', 'crd_lst_agrmnt_osb_cd', 'crd_lst_agrmnt_vsp_cd', 'crd_dc_ever_nflag', 'crd_cc_ever_nflag', 'srv_thanks_nflag', 'srv_mb_nflag', 'srv_mb_tot_qty', 'srv_mb_full_qty', 'srv_mb_econ_qty', 'srv_mb_tot_act_qty', 'srv_mb_trx_1m_amt', 'srv_mb_trx_1m_qty', 'srv_mb_trx_3m_amt', 'srv_mb_trx_3m_qty', 'srv_ap_cell_nflag', 'srv_ap_cell_qty', 'srv_ap_cell_txn_amt', 'srv_ap_cell_txn_qty', 'srv_ap_utl_nflag', 'srv_ap_utl_qty', 'srv_ap_utl_txn_amt', 'srv_ap_utl_txn_qty', 'srv_ap_tlc_nflag', 'srv_ap_tlc_qty', 'srv_ap_tlc_txn_amt', 'srv_ap_tlc_txn_qty', 'srv_ap_loan_nflag', 'srv_ap_loan_qty', 'srv_ap_loan_txn_amt', 'srv_ap_loan_txn_qty', 'srv_ap_budg_nflag', 'srv_ap_budg_qty', 'srv_ap_budg_txn_amt', 'srv_ap_budg_txn_qty', 'srv_ap_othr_nflag', 'srv_ap_othr_qty', 'srv_ap_othr_txn_amt', 'srv_ap_othr_txn_qty', 'srv_ap_cnbx_nflag', 'srv_ap_cnbx_qty', 'srv_ap_cnbx_txn_amt', 'srv_ap_cnbx_txn_qty', 'srv_ap_p2p_nflag', 'srv_ap_p2p_qty', 'srv_ap_p2p_txn_amt', 'srv_ap_p2p_txn_qty', 'srv_sbol_txn_1m_amt', 'srv_sbol_txn_1m_qty', 'srv_sbol_txn_3m_amt', 'srv_sbol_txn_3m_qty', 'srv_sbol_txn_web_1m_amt', 'srv_sbol_txn_web_1m_qty', 'srv_sbol_txn_web_3m_amt', 'srv_sbol_txn_web_3m_qty', 'srv_sbol_txn_mob_1m_amt', 'srv_sbol_txn_mob_1m_qty', 'srv_sbol_txn_mob_3m_amt', 'srv_sbol_txn_mob_3m_qty', 'srv_sbol_txn_atm_1m_amt', 'srv_sbol_txn_atm_1m_qty', 'srv_sbol_txn_atm_3m_amt', 'srv_sbol_txn_atm_3m_qty', 'srv_udbo_nflag', 'tp_active_nflag', 'tp_ever_nflag', 'tp_active_kind_cd', 'tp_best_kind_ever_cd', 'tp_lst_tb_cd', 'tp_lst_kind_cd', 'prd_ib_active_1m_nflag', 'prd_mb_active_1m_nflag', 'prd_ap_cell_active_1m_nflag', 'prd_ap_utl_active_1m_nflag', 'prd_ap_tlc_active_1m_nflag', 'prd_ap_loan_active_1m_nflag', 'prd_ap_budg_active_1m_nflag', 'prd_ap_othr_active_1m_nflag', 'prd_ap_cnbx_active_1m_nflag', 'prd_ap_p2p_active_1m_nflag', 'prd_client_active_1m_nflag', 'prd_client_active_prod_1m_qty', 'lne_tot_agr_open_qty', 'lne_tot_agr_os_qty', 'lne_tot_agr_open_ever_qty', 'lne_tot_agr_clsd_ever_qty', 'lne_tot_agr_clsd_00_ever_qty', 'lne_tot_debt_due_rub_amt', 'lne_tot_debt_ovrd_rub_amt', 'lne_tot_debt_os_rub_amt', 'lne_tot_issued_open_rub_amt', 'lne_tot_issued_ever_rub_amt', 'lne_tot_max_ovrd_open_days', 'lne_tot_max_ovrd_ever_days', 'lne_tot_max_ovrd_open_rub_amt', 'lne_tot_max_ovrd_ever_rub_amt', 'lne_tot_dq_entr_ever_qty', 'lne_tot_open_wavg_intr_rate', 'lne_pl_agr_open_qty', 'lne_pl_agr_os_qty', 'lne_pl_agr_open_ever_qty', 'lne_pl_agr_clsd_ever_qty', 'lne_pl_agr_clsd_00_ever_qty', 'lne_pl_debt_due_rub_amt', 'lne_pl_debt_ovrd_rub_amt', 'lne_pl_debt_os_rub_amt', 'lne_pl_issued_open_rub_amt', 'lne_pl_issued_ever_rub_amt', 'lne_pl_max_ovrd_open_days', 'lne_pl_max_ovrd_ever_days', 'lne_pl_max_ovrd_open_rub_amt', 'lne_pl_max_ovrd_ever_rub_amt', 'lne_pl_dq_entr_ever_qty', 'lne_pl_open_wavg_intr_rate', 'lne_mg_agr_open_qty', 'lne_mg_agr_os_qty', 'lne_mg_agr_open_ever_qty', 'lne_mg_agr_clsd_ever_qty', 'lne_mg_agr_clsd_00_ever_qty', 'lne_mg_debt_due_rub_amt', 'lne_mg_debt_ovrd_rub_amt', 'lne_mg_debt_os_rub_amt', 'lne_mg_issued_open_rub_amt', 'lne_mg_issued_ever_rub_amt', 'lne_mg_max_ovrd_open_days', 'lne_mg_max_ovrd_ever_days', 'lne_mg_max_ovrd_open_rub_amt', 'lne_mg_max_ovrd_ever_rub_amt', 'lne_mg_dq_entr_ever_qty', 'lne_mg_open_wavg_intr_rate', 'crd_cc_txn_spend_12m_qty', 'pnl_oi_total_amt', 'pnl_oi_dc_amt', 'pnl_oi_cc_amt', 'pnl_oi_pl_amt', 'pnl_oi_ml_amt', 'pnl_oi_td_amt', 'pnl_oi_ca_amt', 'pnl_oi_othr_amt', 'crd_txn_spnd_3m_qty', 'crd_txn_spnd_3m_amt', 'crd_txn_snpd_pos_ever_qty', 'prd_mf_active_nflag', 'seg_service_channel_cd', 'inv_mf_agrmnt_open_qty', 'inv_mf_agrmnt_bal_qty', 'inv_mf_agrmnt_bal_rub_amt', 'inv_tm_agrmnt_open_qty', 'inv_tm_agrmnt_bal_qty', 'inv_tm_agrmnt_bal_rub_amt', 'inv_bo_agrmnt_open_qty', 'inv_bo_agrmnt_bal_qty', 'inv_bo_agrmnt_bal_tot_rub_amt', 'inv_bo_agrmnt_bal_sec_rub_amt', 'inv_bo_agrmnt_bal_cash_rub_amt', 'crd_otf_pos_atm_rub_amt', 'crd_otf_pos_atm_qty', 'inv_tot_bal_rub_amt', 'prd_mnth_lst_prod_open_qty', 'prd_mnth_lst_prod_close_qty', 'prd_mnth_1st_prod_open_qty', 'crd_dc_mnth_payroll_open_qty', 'crd_dc_mnth_social_open_qty', 'crd_dc_mnth_payroll_act_qty', 'crd_dc_mnth_social_act_qty', 'crd_dc_mnth_snc_open_qty', 'crd_cc_mnth_snc_issue_qty', 'crd_cc_mnth_snc_close_qty', 'sd_age_mnth_comp_nv', 'prd_yrs_lst_open_qty', 'srv_thanks_mnth_qty', 'srv_mb_mnth_1st_txn_qty', 'srv_mb_mnth_lst_txn_qty', 'srv_ap_cell_mnth_1st_txn_qty', 'srv_ap_cell_mnth_lst_txn_qty', 'srv_ap_utl_mnth_1st_txn_qty', 'srv_ap_utl_mnth_lst_txn_qty', 'srv_ap_tlc_mnth_1st_txn_qty', 'srv_ap_tlc_mnth_lst_txn_qty', 'srv_ap_loan_mnth_1st_txn_qty', 'srv_ap_loan_mnth_lst_txn_qty', 'srv_ap_budg_mnth_1st_txn_qty', 'srv_ap_budg_mnth_lst_txn_qty', 'srv_ap_othr_mnth_1st_txn_qty', 'srv_ap_othr_mnth_lst_txn_qty', 'srv_sbol_mnth_1st_login_qty', 'srv_sbol_mnth_lst_login_qty', 'srv_sbol_mnth_1st_txn_qty', 'srv_sbol_mnth_lst_txn_qty', 'srv_sbol_mnth_web_lst_log_qty', 'srv_sbol_mnth_mob_lst_log_qty', 'srv_sbol_mnth_atm_lst_log_qty', 'srv_sbol_1m_login_nflag', 'srv_sbol_3m_login_nflag', 'srv_sbol_web_1m_login_nflag', 'srv_sbol_web_3m_login_nflag', 'srv_sbol_mob_1m_login_nflag', 'srv_sbol_mob_3m_login_nflag', 'srv_sbol_atm_1m_login_nflag', 'srv_sbol_atm_3m_login_nflag', 'tp_mnth_1st_open_qty', 'tp_mnth_lst_open_qty', 'tp_mnth_lst_grace_end_exp_qty', 'tp_mnth_lst_close_qty', 'crd_open_nflag', 'crd_ever_nflag', 'lne_debt_overdue_bal_rub_amt', 'cla_all_active_nflag', 'lne_tot_debt_int_ovrd_rub_amt', 'crd_otf_cash_atm_rub_amt', 'crd_otf_cash_atm_qty', 'crd_cc_lst_cred_lim_amt', 'crd_cc_lst_ips_cd', 'crd_cc_lst_os_amt', 'crd_cc_mnth_snc_lst_open_qty', 'crd_cc_open_mc_nflag', 'crd_cc_open_mir_nflag', 'crd_cc_open_visa_nflag', 'crd_dc_lst_bal_rub_amt', 'crd_dc_lst_ips_cd', 'crd_dc_mnth_snc_lst_open_qty', 'crd_dc_open_mc_nflag', 'crd_dc_open_mir_nflag', 'crd_dc_open_visa_nflag', 'crd_dc_tot_bal_rub_amt', 'crd_pos_auto_3m_qty', 'crd_pos_auto_rub_3m_amt', 'crd_pos_beuaty_3m_qty', 'crd_pos_beuaty_rub_3m_amt', 'crd_pos_clothes_3m_qty', 'crd_pos_clothes_rub_3m_amt', 'crd_pos_dept_stores_3m_qty', 'crd_pos_dept_stores_rub_3m_amt', 'crd_pos_eat_out_3m_qty', 'crd_pos_eat_out_rub_3m_amt', 'crd_pos_groceries_3m_qty', 'crd_pos_groceries_rub_3m_amt', 'crd_pos_healthcare_3m_qty', 'crd_pos_healthcare_rub_3m_amt', 'crd_pos_home_repair_3m_qty', 'crd_pos_home_repair_rub_3m_amt', 'crd_pos_leisure_3m_qty', 'crd_pos_leisure_rub_3m_amt', 'crd_pos_money_trf_3m_qty', 'crd_pos_money_trf_rub_3m_amt', 'crd_pos_othr_3m_amt', 'crd_pos_othr_3m_qty', 'crd_pos_pc_it_3m_qty', 'crd_pos_pc_it_rub_3m_amt', 'crd_pos_special_3m_qty', 'crd_pos_special_rub_3m_amt', 'crd_pos_telecom_3m_qty', 'crd_pos_telecom_rub_3m_amt', 'crd_pos_tourism_3m_qty', 'crd_pos_tourism_rub_3m_amt', 'crd_pos_utilities_3m_qty', 'crd_pos_utilities_rub_3m_amt', 'crd_trx_cc_cash_3m_amt', 'crd_trx_cc_cash_3m_qty', 'crd_trx_cc_cash_6m_amt', 'crd_trx_cc_cash_6m_qty', 'crd_trx_cc_pos_rtrn_3m_amt', 'crd_trx_cc_pos_rtrn_3m_qty', 'crd_trx_cc_pos_rtrn_6m_amt', 'crd_trx_cc_pos_rtrn_6m_qty', 'crd_trx_cc_pos_spnd_3m_amt', 'crd_trx_cc_pos_spnd_3m_qty', 'crd_trx_cc_pos_spnd_6m_amt', 'crd_trx_cc_pos_spnd_6m_qty', 'crd_trx_dc_cash_3m_amt', 'crd_trx_dc_cash_3m_qty', 'crd_trx_dc_cash_6m_amt', 'crd_trx_dc_cash_6m_qty', 'crd_trx_dc_cash_6m_spnd_amt', 'crd_trx_dc_cash_6m_spnd_qty', 'crd_trx_dc_pos_rtrn_3m_amt', 'crd_trx_dc_pos_rtrn_3m_qty', 'crd_trx_dc_pos_rtrn_6m_amt', 'crd_trx_dc_pos_rtrn_6m_qty', 'crd_trx_dc_pos_spnd_3m_amt', 'crd_trx_dc_pos_spnd_3m_qty', 'crd_trx_dc_pos_spnd_6m_amt', 'crd_trx_dc_pos_spnd_6m_qty', 'crd_trx_spnd_cash_avg_3m_amt', 'crd_trx_spnd_pos_avg_3m_amt', 'dep_pop_act_nflag', 'dep_pop_exp_2m_bal_rub_amt', 'dep_pop_exp_2m_nflag', 'dep_pop_exp_3m_bal_rub_amt', 'dep_pop_exp_3m_nflag', 'dep_pop_tot_bal_rub_amt', 'dep_sohr_exp_2m_bal_rub_amt', 'dep_sohr_exp_2m_nflag', 'dep_sohr_exp_3m_bal_rub_amt', 'dep_sohr_exp_3m_nflag', 'dep_topup_12m_avg_rub_amt', 'dep_topup_12m_qty', 'dep_tot_exp_2m_bal_rub_amt', 'dep_tot_exp_3m_bal_rub_amt', 'dep_upr_exp_2m_bal_rub_amt', 'dep_upr_exp_2m_init_rub_amt', 'dep_upr_exp_2m_nflag', 'dep_upr_exp_3m_nflag', 'dep_upr_tot_bal_rub_amt', 'seg_crd_segm_special_cd', 'seg_crd_segm_special_prev_cd', 'seg_crm_day_qty', 'seg_crm_spec_year_qty', 'seg_crm_year_qty', 'seg_premier_cd', 'srv_mb_quick_service_nflag', 'srv_mb_trx_cell_3m_qty', 'srv_mb_trx_p2p_3m_qty', 'dep_social_client_nflag', 'sd_dead_nflag', 'lne_loan_overdue_nflag', 'prd_crd_cc_active_1m_nflag', 'prd_crd_dc_active_1m_nflag', 'prd_pay_active_1m_nflag', 'prd_mb_active_1m_nflag', 'prd_mb_active_nflag', 'prd_crd_cc_active_nflag', 'sd_cb_staff_nflag', 'prd_p2p_active_nflag', 'sd_name_age_segment_cd', 'sd_stlmnt_type_cd', 'seg_client_mp_segment_cd', 'lne_overdraft_overdue_nflag', 'dep_payroll_client_nflag', 'prd_p2p_active_1m_nflag', 'seg_age_segment', 'prd_pay_active_nflag', 'prd_crd_dc_active_nflag', 'sd_gender_cd', 'seg_crm_segment_cd', 'prd_lst_prod_tb_cd', 'prd_lst_prod_cd', 'prd_1st_prod_tb_cd', 'prd_1st_prod_cd', 'prd_client_type_final_cd', 'crd_lst_agrmnt_tb_cd', 'crd_lst_agrmnt_osb_cd', 'crd_lst_agrmnt_vsp_cd', 'tp_active_kind_cd', 'tp_best_kind_ever_cd', 'tp_lst_tb_cd', 'tp_lst_kind_cd', 'seg_service_channel_cd', 'crd_cc_lst_ips_cd', 'crd_dc_lst_ips_cd', 'seg_crd_segm_special_cd', 'seg_crd_segm_special_prev_cd', 'seg_premier_cd', 'sd_client_valid_nflag', 'sd_doc_valid_nflag', 'sd_russian_citizen_nflag', 'sd_resident_nflag', 'sd_sbrf_employee_nflag', 'seg_private_banking_nflag', 'prd_ma_active_nflag', 'prd_da_active_nflag', 'prd_sc_active_nflag', 'prd_pl_active_nflag', 'prd_mg_active_nflag', 'prd_ib_active_nflag', 'prd_ap_cell_active_nflag', 'prd_ap_utl_active_nflag', 'prd_ap_tlc_active_nflag', 'prd_ap_loan_active_nflag', 'prd_ap_budg_active_nflag', 'prd_ap_othr_active_nflag', 'prd_ap_cnbx_active_nflag', 'prd_ap_p2p_active_nflag', 'prd_client_active_nflag', 'crd_dc_payroll_nflag', 'crd_dc_payroll_pmt_nflag', 'crd_dc_payroll_act_nflag', 'crd_dc_payroll_sbrf_act_nflag', 'crd_dc_social_nflag', 'crd_dc_social_pmt_nflag', 'crd_dc_social_act_nflag', 'crd_cc_overdue_nflag', 'crd_dc_ever_nflag', 'crd_cc_ever_nflag', 'srv_thanks_nflag', 'srv_mb_nflag', 'srv_ap_cell_nflag', 'srv_ap_utl_nflag', 'srv_ap_tlc_nflag', 'srv_ap_loan_nflag', 'srv_ap_budg_nflag', 'srv_ap_othr_nflag', 'srv_ap_cnbx_nflag', 'srv_ap_p2p_nflag', 'srv_udbo_nflag', 'tp_active_nflag', 'tp_ever_nflag', 'prd_ib_active_1m_nflag', 'prd_ap_cell_active_1m_nflag', 'prd_ap_utl_active_1m_nflag', 'prd_ap_tlc_active_1m_nflag', 'prd_ap_loan_active_1m_nflag', 'prd_ap_budg_active_1m_nflag', 'prd_ap_othr_active_1m_nflag', 'prd_ap_cnbx_active_1m_nflag', 'prd_ap_p2p_active_1m_nflag', 'prd_client_active_1m_nflag', 'prd_mf_active_nflag', 'srv_sbol_1m_login_nflag', 'srv_sbol_3m_login_nflag', 'srv_sbol_web_1m_login_nflag', 'srv_sbol_web_3m_login_nflag', 'srv_sbol_mob_1m_login_nflag', 'srv_sbol_mob_3m_login_nflag', 'srv_sbol_atm_1m_login_nflag', 'srv_sbol_atm_3m_login_nflag', 'crd_open_nflag', 'crd_ever_nflag', 'cla_all_active_nflag', 'crd_cc_open_mc_nflag', 'crd_cc_open_mir_nflag', 'crd_cc_open_visa_nflag', 'crd_dc_open_mc_nflag', 'crd_dc_open_mir_nflag', 'crd_dc_open_visa_nflag', 'dep_pop_act_nflag', 'dep_pop_exp_2m_nflag', 'dep_pop_exp_3m_nflag', 'dep_sohr_exp_2m_nflag', 'dep_sohr_exp_3m_nflag', 'dep_upr_exp_2m_nflag', 'dep_upr_exp_3m_nflag', 'srv_mb_quick_service_nflag','crd_inf_total_qty', 'crd_inf_total_qty_3a', 'crd_inf_total_qty_6a', 'crd_inf_total_qty_12a',  'crd_otf_total_qty', 'crd_otf_total_qty_3a', 'crd_otf_total_qty_6a', 'crd_otf_total_qty_12a']

In [10]:
config['hue_db'] = 'qg_td' #hive-бд с датасетами, собранными инженерами
config['hue_db_for_traning'] = 'cltv_refit_2021' #cюда будем писать енкоженные датасеты

config.update({
    #hue objects
    'hue_obj_target': config['hue_db'] + '.cltv_target_type{}_{}_{}',
    'hue_obj_features': config['hue_db'] +'.cltv_feature_{}_{}',
    'data_path': '/opt/workspace/yakovlev-ra_ca-sbrf-ru/cltv_final_refit/Data',
    'base_products': [
        'ap', 'cc', 'ch', 'd33', 'dc1', 'dc2p', 'dep', 'dlcc', 'dnpr', 'h', 'hu', 'iis', 'mapp', 'mb',
        'ph', 'pl', 'plg', 'pt', 'sbl', 'sc', 'sf', 'sh', 'sm', 'st', 't2f', 'th', 'okko'
    ],
    "target_name": 'TRG',
    'id_col': 'client_dk'
})

In [ ]:
nd_path = os.path.join(os.path.expanduser('~'), 'cltv_refit_2021')
art_path = os.path.join(nd_path, 'data')
[os.mkdir(x) for x in (nd_path, art_path) if not os.path.exists(x)]



mapping_df = pd.read_excel(
    f"{nd_path}Copy of Маппинг переименований v2.xlsx"
).drop_duplicates('Исходные атрибуты новой витрины')

nd_cols = mapping_df['Spark SQL Expr'].tolist()
nd_cols = [eval(re.sub(r'\\|,', '', x)) for x in nd_cols]

joblib.dump(nd_cols, f'{nd_path}feature_transform_select_expr.pkl')

remane_dict = dict(zip(*[mapping_df[x].tolist() for x in ('Атрибут в старой витрине', 'Исходные атрибуты новой витрины')]))
remane_dict = {x: y for x, y in remane_dict.items() if str(y) != 'nan'}



In [37]:
%%time


cols_uniq_all_path = os.path.join(nd_path, 'uniq_ctg_features_20200831_3y.pkl')
tr_exp_path = os.path.join(nd_path, 'select_exp_train_data_tedadata_datasets.pkl')

spark.sql(f'use {config["hue_db"]}')
tabs = [
    x[0] for x in 
    spark.sql("show tables like 'cltv_feature_%_trai%' ")[['tablename']].collect()
]


if not any([os.path.exists(x) for x in (cols_uniq_all_path, tr_exp_path)]):
    cols_uniq_all, tr_exp = {}, []
    
    for bp in tqdm(config['base_products']):
        
        tab = config['hue_obj_features'].format(bp, 'train').split('.')[-1]
        
        if tab not in tabs:
            print(f'couldn\'t find {tab} in {config["hue_db"]}')
            continue
        df = r(f'{config["hue_db"]}.{tab}').drop(*config['drop_features'])
        #renaming

        nd_cols_dataset = []

        for cur_col in (set(df.columns)&set(config['categorical_features']))-set(config['drop_features']):
            added_cols = [x for x in nd_cols if cur_col.lower() in x.lower().split(' as ')[0]]
            nd_cols_dataset.extend(added_cols)

        nd_cols_dataset.extend([
            x for x in config['categorical_features'] 
            if x.lower() not in mapping_df['Атрибут в старой витрине'].apply(lambda x: str(x).lower())
            and x.lower() not in mapping_df['Исходные атрибуты новой витрины'].apply(lambda x: str(x).lower())
            and len([y for y in nd_cols_dataset if x.lower() in y.lower().split(' as ')[-1]]) == 0
            and x not in config['drop_features']
            and x in df.columns
        ])
        tr_exp.extend([
            x for x in nd_cols_dataset if x not in tr_exp
        ])
        df = df.selectExpr(*nd_cols_dataset)
        joblib.dump(nd_cols_dataset, os.path.join(art_path, f'select_exp_train_data_{bp}.pkl'))

        cols_uniq = [
            x.asDict() for x in 
            df.select([F.collect_set(x).alias(x) for x in df.columns]).collect()
        ][0]

        for col in cols_uniq.keys():
            if len(cols_uniq[col]) > 1000:
                print(f'suspicious big categ: {col}: {len(cols_uniq[col])}')


        if len(cols_uniq_all.keys()) == 0:
            cols_uniq_all.update(cols_uniq)
        else:
            for col in cols_uniq.keys():
                if col in cols_uniq_all.keys():
                    cols_uniq_all[col] += \
                        list(filter(lambda x: x not in cols_uniq_all[col], cols_uniq[col]))
                else:
                    cols_uniq_all[col] = cols_uniq[col]
    joblib.dump(cols_uniq_all_path, cols_uniq_all_path)
    joblib.dump(tr_exp, tr_exp_path)
else:
    cols_uniq_all = joblib.load(cols_uniq_all_path)
    tr_exp = joblib.load(tr_exp_path)


NameError: name 'spark' is not defined

In [39]:
%%time

trainsformer_path = os.path.join(nd_path, 'cltv_transformer_full_ld_20210113_3y.pkl')

if not os.path.exists(trainsformer_path):
    
    cols_uniq_all['Product'] = config['base_products']

    for col in cols_uniq_all.keys():
        cols_uniq_all[col].extend(
            list(map(
                lambda x: x.replace(' ', '').lower(),
                [x for x in cols_uniq_all[col] if ' ' in str(x)]
        )))

    for col in cols_uniq_all.keys():
        cols_uniq_all[col].extend([None, np.nan,'', ' '])

    with open(cols_uniq_all_path, 'wb') as f:
        pickle.dump(cols_uniq_all, f)
    
    uniq_df = pd.DataFrame.from_dict(cols_uniq_all, orient = 'index').T
    
    _, transformer = transformers.encode_categorical(
        config = {'categorical_features': list(cols_uniq_all.keys())}, 
        train = (uniq_df, None),
    )
    with open(trainsformer_path, 'wb') as f:
        pickle.dump(transformer, f)

else:
    transformer = joblib.load(trainsformer_path)
    
sc.addPyFile(trainsformer_path)


NameError: name 'joblib' is not defined

##### preraping targets

In [8]:
from typing import Dict

def replace_all(text: pd.Series, dic: Dict) -> pd.Series:
    for i, j in dic.items():
        text = text.str.replace(i, j)
    return text

def get_map(range_, step = 3) -> dict:
    out = {}
    f = 1
    if step > 0:
        for i in range(range_[0], range_[-1], step):
            for j in list(map(lambda x: i + x, range(step))):
                if j == 0:
                    out[f't{j}'] = 't0'
                else:
                    out[f't{j}'] = f't{f}'
            f += 1
        return out
    else:
        out = [f't{x}' for x in range(range_[0], range_[-1])]
        return dict(zip(out, out))

def drop_(string):
    while string.endswith('_'):
        string = string[:-1]
    return string

def match(x, list_of_values, protected_values = False):
    for val in list_of_values:
        if val.lower() in x.lower():
            if not protected_values:
                return True
            else:
                for pr_val in protected_values:
                    if pr_val.lower() in x.lower():
                        return False
                return True
    return False

def go_mapping(x: str) -> str:
    product, t_per = x.split('_')
    t_per = mapping.get(t_per)
    return '_'.join([product, t_per])


def split_string_with_product(value):
    val_1, val_2 = value.split('_t')
    val_2 = int(val_2)
    return T.Row('Product', 'T')(val_1, val_2)

split_string_udf = F.UserDefinedFunction(
    split_string_with_product, 
    returnType = T.StructType([
        T.StructField('Product', T.StringType()),
        T.StructField('T', T.IntegerType())
    ]))

def transform_spark_df(sdf, table_name):
    transform_column_pattern = '''@F.pandas_udf('bigint')
def transform_column_{col}(*df_col):
    unpack_dspl()
    transformer = joblib.load(trainsformer_path.split('/')[-1])
    df_col = pd.DataFrame(df_col[0])
    df_col.columns = ['{col}']
    df_col = transformer.transform(df_col)
    return df_col.iloc[:,0].astype('int64')'''
    
    for categ_col in [x for x in sdf.columns if x in transformer.cat_features]:
        tr_f_name = f'transform_column_{categ_col}'
        #для спарка функции должны быть глобалками, тк они и все их компоненты пиклятся из main
        #и ледят в таком виде на распределенные екзекьюторы для выполнения
        exec(f'global {tr_f_name}\n' + transform_column_pattern.format(col=categ_col))
        sdf = sdf.withColumn(
            categ_col, 
            eval(tr_f_name)(*[F.regexp_replace(F.lower(F.col(categ_col)), ' ', '')])
        )
    if 'Product' in sdf.columns:
        prod_partition, tab_partition = '.repartition(F.col("Product"))', '.partitionBy("Product")'
    else:
        prod_partition, tab_partition = '.repartition(6)', ''
    drop_hive_table(table_name)
    
    exec(f'''sdf{prod_partition}\
        .write\
        .option('compression', 'gzip'){tab_partition}\
        .mode('overwrite')\
        .saveAsTable(table_name)
        ''')

In [10]:
config['hue_db'] = 'qg_td'
config['hue_db_for_traning'] = 'cltv_refit_2021'

config.update({
    #hue objects
    'hue_obj_target': config['hue_db'] + '.cltv_target_type{}_{}_{}',
    'hue_obj_features': config['hue_db'] +'.cltv_feature_{}_{}',
    'data_path': '/opt/workspace/yakovlev-ra_ca-sbrf-ru/cltv_final_refit/Data',
    'base_products': ['okko'],# 
    #done: 'dep', 'dnpr', 'ap', 'mapp', 'plg', 'mb', 'h' 'cc', 'dc', 'pl', 'th', 'iis', 't2f', 'sh', 'sf', 'd33', 'dlcc', 'raso'
    #'pt', 'ph', 'st', 'sm', 'sc', 'ch','EL14', 'EL11', 'EL17', 'EL18', 'EL16', 'EL19', 'EL0', 'EL15',  'EL12', 'EL13'
    # 'dnpr2'
    "target_name": 'TRG',
    'id_col': 'client_dk'
})

In [11]:
mapping = get_map((0, 35))
print(mapping)

{'t0': 't0', 't1': 't1', 't2': 't1', 't3': 't2', 't4': 't2', 't5': 't2', 't6': 't3', 't7': 't3', 't8': 't3', 't9': 't4', 't10': 't4', 't11': 't4', 't12': 't5', 't13': 't5', 't14': 't5', 't15': 't6', 't16': 't6', 't17': 't6', 't18': 't7', 't19': 't7', 't20': 't7', 't21': 't8', 't22': 't8', 't23': 't8', 't24': 't9', 't25': 't9', 't26': 't9', 't27': 't10', 't28': 't10', 't29': 't10', 't30': 't11', 't31': 't11', 't32': 't11', 't33': 't12', 't34': 't12', 't35': 't12'}


send_beeline_query(f'use {config["hue_db"]};show tables like \'%\';')

In [12]:
%%time

rebuild = False

samples = ('train','oot')

for bp in config["base_products"]: 
    
    for _ in ('',):
        
        print(f'rebuilding targets for {bp}')

        config["base_product"] = bp
        
        tab_name_mask = f'{config["hue_db_for_traning"]}.targets_{config["base_product"]}_'
        
        if all([
            check_hive_table_existence(x) for x in
            [tab_name_mask + sample for sample in samples]
        ]) and not rebuild: continue        

        spark.sql(f'use {config["hue_db"]}')

        raw_targets = sorted([x.asDict()['tableName'] for x in spark.sql(f'''
            show tables like '%target\_type%\_{config['base_product']}\_%'
        ''')[['tableName']].collect()])

        raw_targets = [int(re.findall(r'type\d', x)[0][-1]) for x in raw_targets]
        config['type_target_range'] = [min(raw_targets), max(raw_targets)]    
        
        print(f'target time range: {config["type_target_range"]}')
        
        sets = {}

        for sample in samples:
            targets = []
            for i in range(config['type_target_range'][0], config['type_target_range'][-1] + 1):
                params = i, config['base_product'], sample
                table_name = config['hue_obj_target'].format(*params)
                targets.append(spark.sql(f'select * from {table_name}'))
            sets[sample] = targets

        for sample in samples: #

            tab_name = tab_name_mask + sample

            if check_hive_table_existence(tab_name) and not rebuild:
                print(f'{tab_name} already exists')
                continue

            some_range = len(sets[sample])

            for i in range(some_range):

                to_drop = [
                    x for x in sets[sample][i].columns 
                    if match(x, ['gld', 'oi', 'pp', 'sale_month', 'lag_t'], ['mapp'])
                ]
                
                to_drop += [x for x in sets[sample][i].columns if 't36' in x]
                
                if 'sale_month' in sets[sample][i].columns:
                    sets[sample][i] = \
                        sets[sample][i].withColumnRenamed('sale_flg', config['base_product'])#.drop('product_cd')

                sets[sample][i] = sets[sample][i].drop(*to_drop)
                
                for col in sets[sample][i].columns:
                    if '_new' in col:
                        sets[sample][i] = sets[sample][i].drop(col.replace('_new', ''))
                        sets[sample][i] = sets[sample][i].withColumnRenamed(
                            col, col.replace('_new', '')
                        )
                #схлопываем t по кварталам, согласно mapping
                t_cols = [x for x in sets[sample][i].columns if '_t' in x]
                const_cols = list(map(drop_, set(sets[sample][i].columns) ^ set(t_cols)))
                new_t_cols = pd.DataFrame(zip(t_cols, list(map(go_mapping, t_cols))))
                new_t_cols = new_t_cols.groupby(1).agg({0: list})[0].to_dict()
                new_t_cols_names = list(new_t_cols.keys())
                new_t_cols = [
                    f'max(case '\
                    f'when {"+".join(new_t_cols[x])} > 0 then 1 '\
                    f'when {"+".join(new_t_cols[x])} = 0 then 0 '\
                    f'else null end) as {x}' 
                    for x in new_t_cols.keys()
                    ]
                sets[sample][i].createOrReplaceTempView(f'df_{i}')
                sets[sample][i] = \
                    spark.sql(f'''
                        select 
                              {','.join(const_cols)}
                             ,{','.join(new_t_cols)}
                        from df_{i}
                        group by {','.join(const_cols)}
                    ''')

                spark.sql(f'drop view df_{i}')
                
                #выполняем разворот по паре продукт_время
                vals_for_stack = [f'\'{x}\', {x}' for x in new_t_cols_names]
                sets[sample][i] = \
                    sets[sample][i].selectExpr(
                        *const_cols,
                        f"stack({len(new_t_cols_names)}, {','.join(vals_for_stack)}) as (Product_T, {config['target_name']})"
                    )

                sets[sample][i] = sets[sample][i].withColumn(
                    'split_string', split_string_udf(F.col('Product_T'))
                )

                sets[sample][i] = \
                    sets[sample][i]\
                        .selectExpr('*', 'split_string.Product')\
                        .selectExpr('*', 'split_string.T')\
                        .drop(*['split_string', 'Product_T'])
                
                date_cols = [x[0] for x in sets[sample][i].dtypes if 'date' in x[1].lower()]
                for col in date_cols:
                    sets[sample][i] = sets[sample][i].withColumn(
                        col, F.col(col).cast(T.TimestampType())
                    )
            t_cols = sets[sample][0].columns
            sets[sample] = eval('sets[sample][0]' + (lambda: some_range > 1 and '.' or '')()\
                                + ".".join([f'unionAll(sets[sample][{x}].select(t_cols))' for x in range(1, i + 1)]))

            sets[sample] = sets[sample].withColumn(
                config['base_product'], F.lower(F.col(config['base_product']))
            )

            spark.sql(f'drop table if exists {tab_name}')

            for column in sets[sample].dtypes:
                if 'date' in column[1]:
                    sets[sample] = sets[sample].withColumn(
                        column[0], F.col(column[0]).cast(T.TimestampType()).alias(column[0])
                    )
            
            print(f'saving {tab_name}')
            
            sets[sample].withColumn('report_dt', F.col('report_dt').cast(T.TimestampType()))\
                .filter(F.col(config['target_name']).isNotNull())\
                .repartition(*[F.col('Product')])\
                .write\
                .option('compression', 'gzip')\
                .partitionBy('Product')\
                .mode('overwrite')\
                .saveAsTable(tab_name)

            print(f'{tab_name} rdy')
        
    spark.catalog.clearCache()
    
    print(f'transforming {bp}')

    config["base_product"] = bp
        
    datasets = {
        'train_feature': r(config['hue_obj_features'].format(bp, 'train')),
        'train_target': r(f'{config["hue_db_for_traning"]}.targets_{bp}_train'),
        'oot_feature': r(config['hue_obj_features'].format(bp, 'oot')),
        'oot_target': r(f'{config["hue_db_for_traning"]}.targets_{bp}_oot'),
    }
    
    tr_exp_p = [
        x for x in tr_exp if x.lower().split(' as ')[-1] not in datasets['train_feature'].columns
    ]
    
    datasets.update({
        x: datasets[x].selectExpr(*['*', *tr_exp_p])
        for x in [ds for ds in datasets.keys() if 'feature' in ds]
    })
    
    for dataset in datasets.keys():
        print(f'{dataset}: {datasets[dataset].select(config["id_col"]).distinct().count()}')
    
    #проверяем, что таргеты джойнятся к фичам
    if len(samples) > 1:
        print(
            'joined {} ->\ttrain: {}, oot: {}'\
                .format(*[bp] + [
                    datasets[f'{sample}_feature']\
                        .join(datasets[f'{sample}_target'], on = 'client_dk')\
                        .select('client_dk')\
                        .distinct()\
                        .count()
                    for sample in samples
        ]))

    #удаляем фичи, более 50% которых составляют null
    def count_null_percent(sp_df, limit = 0.5):
        all_rows = sp_df.count()
        null_cols_cnt = get_parquets_from_sdf(
            sp_df.agg(*[
                F.sum(F.isnull(F.col(x)).cast('integer')).alias(x) for x in sp_df.columns
        ]))
        return null_cols_cnt.columns[null_cols_cnt.max() / all_rows > limit].tolist()
    
    not_ld_features = list(filter(
        lambda x: x not in allowed_features + ['client_dk', 'report_dt'],
        datasets['train_feature'].columns
    ))
    
    datasets['train_feature'] = datasets['train_feature'].drop(*not_ld_features)
    
    datasets['train_feature'] = \
        datasets['train_feature'].drop(*count_null_percent(datasets['train_feature']))
    
    if 'oot_feature' in datasets.keys():
        datasets['oot_feature'] = datasets['oot_feature'].select(datasets['train_feature'].columns)
    
    for dataset in datasets.keys():
        
        tab_name = f'{config["hue_db_for_traning"]}.'\
            f'{dataset}_{config["base_product"]}_encoded'
        
        if check_hive_table_existence(tab_name) and not rebuild:
            print(f'{tab_name} already exists')
            datasets[dataset] = read_hive_table(f'{tab_name}')
            continue
        
        drop_hive_table(tab_name, True)
        
        #date надо заменить на timestamp, тк при сохранении в паркет багует
        date_cols = [x[0] for x in datasets[dataset].dtypes if 'date' in x[1].lower()]
        for col in date_cols:
            datasets[dataset] = datasets[dataset].withColumn(
                col, F.col(col).cast(T.TimestampType())
            )
        
        transform_spark_df(sdf = datasets[dataset], table_name = tab_name)

        print(f'{tab_name} transformed')
        spark.catalog.clearCache()
    

rebuilding targets for okko
transforming okko
train_feature: 500000
train_target: 500000
oot_feature: 500000
oot_target: 500000
joined okko ->	train: 500000, oot: 500000
cltv_refit_2021.train_feature_okko_encoded transformed
cltv_refit_2021.train_target_okko_encoded transformed
cltv_refit_2021.oot_feature_okko_encoded transformed
cltv_refit_2021.oot_target_okko_encoded transformed
CPU times: user 2.87 s, sys: 1.71 s, total: 4.57 s
Wall time: 4min 50s


In [13]:
sc.stop()